<a href="https://colab.research.google.com/github/MikaZico36/DECDProject/blob/Carolina/Knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns


%matplotlib inline
import matplotlib.pyplot as plt

In [3]:
import os


numeroContratosContador_df = pd.read_csv('https://raw.githubusercontent.com/CarolinaSS98/project_decd/main/contadores-de-energia-atual.csv', sep =";", low_memory=False)
energiaConsumida_df = pd.read_csv('https://raw.githubusercontent.com/CarolinaSS98/project_decd/main/3-consumos-faturados-por-municipio-ultimos-10-anos.csv', sep =";")
numeroOrdensServico_df = pd.read_csv('https://raw.githubusercontent.com/CarolinaSS98/project_decd/main/15-ordens-de-servico.csv', sep =";")

# numeroContratosContador2023_df = numeroContratosContador_df[numeroContratosContador_df['Ano'] == 2023]
# energiaConsumida2023_df = energiaConsumida_df[energiaConsumida_df['Ano'] == 2023]
# numeroOrdensServico2023_df = numeroOrdensServico_df[numeroOrdensServico_df['Ano'] == 2023]


### Tratar Coleção - Número de Contratos Ativos por Tipo de Contador

#### Agregar dados por "data" e "Concelho"

Os números de CPE's foram separados em duas colunas:

- Número de CPEs=SIM : representa o número de CPE's ***com contadores inteligentes***.
- Número de CPEs=NAO : representa o número de CPE's ***sem contadores inteligentes***.

In [4]:
# Filtrando os dados para separar os CPEs com e sem contador inteligente
df_sim = numeroContratosContador_df[numeroContratosContador_df['Inclui contador inteligente'] == 'SIM']
df_nao = numeroContratosContador_df[numeroContratosContador_df['Inclui contador inteligente'] == 'NÃO']

# Agrupando os dados por data e concelho e contando o número de CPEs para cada grupo
contadoresAgregados_df = pd.DataFrame({
    'Num_CPEs_Inteligentes': df_sim.groupby(['data','Concelho'])["Número de CPE's"].sum(),
    'Num_CPEs_Nao_Inteligentes': df_nao.groupby(['data','Concelho'])["Número de CPE's"].sum()
}).reset_index()

contadoresAgregados_df.rename(columns={'data': 'Data'}, inplace=True)

# Imprimindo o DataFrame resultante
contadoresAgregados_df.head(100)
# contadoresAgregados_df.info()

,Data,Concelho,Num_CPEs_Inteligentes,Num_CPEs_Nao_Inteligentes
0,2022-03,Abrantes,13331,11795
1,2022-03,Aguiar da Beira,2331,2587
2,2022-03,Alandroal,1977,2121
3,2022-03,Albergaria-a-Velha,5596,8525
4,2022-03,Albufeira,28711,24554
...,...,...,...,...
95,2022-03,Freixo de Espada À Cinta,1803,1730
96,2022-03,Fronteira,2057,469
97,2022-03,Fundão,17512,6518
98,2022-03,Gavião,1863,1664


### Tratar Coleção - Consumo de Energia Consumida

####Agregar dados por "data" e "Concelho"

O valor de  Energia Ativa (kWh) foi separado em duas colunas:
- **Energia_Ativa_Baixa_Tensao** : represneta o valor de energia consumida com nível de tensão baixo.
- **Energia_Ativa_Outra_Tensao** : represneta o valor de energia consumida com nível de tensão Média, Alta e Muito Alta Tensão

In [5]:
# Filtrando os dados para separar a Energia Consumida por nível de tensão
df_baixa = energiaConsumida_df[energiaConsumida_df['Nível de Tensão'] == "Baixa Tensão"]
df_acimaBaixa = energiaConsumida_df[energiaConsumida_df['Nível de Tensão'] == "Muito Alta, Alta e Média Tensões"]

# Agrupando os dados por data e concelho e sumando o valor de energia consumida por cada um dos grupos
energiaAgregada_df = pd.DataFrame({
    'Energia_Consumida_Baixa_T': df_baixa.groupby(['Data','Concelho'])['Energia Ativa (kWh)'].sum(),
    'Energia_Consumida_Superior_Baixa_T': df_acimaBaixa.groupby(['Data','Concelho'])['Energia Ativa (kWh)'].sum()
}).reset_index()


# Imprimir infor do DataFrame agregado dos consumos
energiaAgregada_df.head()

,Data,Concelho,Energia_Consumida_Baixa_T,Energia_Consumida_Superior_Baixa_T
0,2020-11,Abrantes,6.028607e+06,9.314171e+06
1,2020-11,Aguiar da Beira,1.065748e+06,2.623619e+05
2,2020-11,Alandroal,1.110645e+06,1.498483e+05
3,2020-11,Albergaria-a-Velha,4.153435e+06,1.144006e+07
4,2020-11,Albufeira,1.534018e+07,5.047671e+06


#### Tratar valores Null Detetados

Dado terem sido detetados valores a NULL nas colunas de Energia Consumida de nível > Baixa Tensão, indica que existem concelhos onde não existe consumo deste nível de tensão.

A forma de tratar estes valores a null é colocá-los com o valor da mediana da coluna.

In [11]:
#mostra quais sao as linhas onde aparecem valores de colunas a NaN
#linhas_valor_null = energiaAgregada_df[energiaAgregada_df['Energia_Consumida_Superior_Baixa_T'].isnull()]
#linhas_valor_null.head()

linhas_not_null = energiaAgregada_df[energiaAgregada_df['Energia_Consumida_Superior_Baixa_T'].notna()]
#linhas_not_null.info()
#linhas_not_null.head()
mediana_acimaBaixa = linhas_not_null['Energia_Consumida_Superior_Baixa_T'].median()
#print(mediana_acimaBaixa)
energiaAgregada_df['Energia_Consumida_Superior_Baixa_T'].fillna(mediana_acimaBaixa, inplace=True)

energiaAgregada_df.info()

# energiaAgregada_df['Energia_Consumida_Superior_Baixa_T'] = energiaAgregada_df['Energia_Consumida_Superior_Baixa_T'].replace(np.nan, 0)

#energiaAgregada_df.info()

# linhas_valor_zero = energiaAgregada_df[energiaAgregada_df['Energia_Consumida_Superior_Baixa_T'] == 0]

# linhas_valor_zero.head(100)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11398 entries, 0 to 11397
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Data                                11398 non-null  object 
 1   Concelho                            11398 non-null  object 
 2   Energia_Consumida_Baixa_T           11398 non-null  float64
 3   Energia_Consumida_Superior_Baixa_T  11398 non-null  float64
dtypes: float64(2), object(2)
memory usage: 356.3+ KB


In [ ]:
energiaAgregada_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11398 entries, 0 to 11397
Data columns (total 4 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Data                                11398 non-null  object 
 1   Concelho                            11398 non-null  object 
 2   Energia_Consumida_Baixa_T           11398 non-null  float64
 3   Energia_Consumida_Superior_Baixa_T  11314 non-null  float64
dtypes: float64(2), object(2)
memory usage: 356.3+ KB


### Tratar Coleção - Nº de Ordens de Serviço

####Agregar dados por "data" e "Concelho"

Não nos pareceu relevante agregar por concelho, contado o numero total de ordens de serviço por tipo e por isso vamos diretamente somar o número de ordens de serviço por concelho, ignorando a sua tipologia

In [12]:
numeroOrdensServicoAgregado_df = pd.DataFrame({
    'Numero_total_ordens_servico': numeroOrdensServico_df.groupby(['data','Concelho'])['Ordens Serviço Realizadas (#)'].sum(),
}).reset_index()

numeroOrdensServicoAgregado_df.rename(columns={'data': 'Data'}, inplace=True)

numeroOrdensServicoAgregado_df.head(100)

numeroOrdensServicoAgregado_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7227 entries, 0 to 7226
Data columns (total 3 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Data                         7227 non-null   object
 1   Concelho                     7227 non-null   object
 2   Numero_total_ordens_servico  7227 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 169.5+ KB


In [13]:
merged_aux = pd.merge(contadoresAgregados_df,energiaAgregada_df, on=['Data','Concelho'], how='inner')
merged_df = pd.merge(merged_aux,numeroOrdensServicoAgregado_df, on=['Data','Concelho'], how='inner')

merged_df.head()

#merged_df.info()


,Data,Concelho,Num_CPEs_Inteligentes,Num_CPEs_Nao_Inteligentes,Energia_Consumida_Baixa_T,Energia_Consumida_Superior_Baixa_T,Numero_total_ordens_servico
0,2022-03,Abrantes,13331,11795,6.376950e+06,7.157803e+06,150
1,2022-03,Aguiar da Beira,2331,2587,1.074397e+06,2.613668e+05,9
2,2022-03,Alandroal,1977,2121,1.258384e+06,2.074845e+05,23
3,2022-03,Albergaria-a-Velha,5596,8525,4.441889e+06,1.406621e+07,52
4,2022-03,Albufeira,28711,24554,1.840090e+07,7.379642e+06,385


In [14]:
merged_df['Ano'] = merged_df.Data.str[:4]
merged_df['Mes'] = merged_df.Data.str[5:]
merged_df = merged_df.drop('Data', axis = 1)

In [15]:
merged_df.Concelho = merged_df.Concelho.astype('category')
merged_df.Mes = merged_df.Mes.astype('category')

In [16]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6825 entries, 0 to 6824
Data columns (total 8 columns):
 #   Column                              Non-Null Count  Dtype   
---  ------                              --------------  -----   
 0   Concelho                            6825 non-null   category
 1   Num_CPEs_Inteligentes               6825 non-null   int64   
 2   Num_CPEs_Nao_Inteligentes           6825 non-null   int64   
 3   Energia_Consumida_Baixa_T           6825 non-null   float64 
 4   Energia_Consumida_Superior_Baixa_T  6825 non-null   float64 
 5   Numero_total_ordens_servico         6825 non-null   int64   
 6   Ano                                 6825 non-null   object  
 7   Mes                                 6825 non-null   category
dtypes: category(2), float64(2), int64(3), object(1)
memory usage: 350.7+ KB


In [17]:
merged_df.head(100)

,Concelho,Num_CPEs_Inteligentes,Num_CPEs_Nao_Inteligentes,Energia_Consumida_Baixa_T,Energia_Consumida_Superior_Baixa_T,Numero_total_ordens_servico,Ano,Mes
0,Abrantes,13331,11795,6.376950e+06,7.157803e+06,150,2022,03
1,Aguiar da Beira,2331,2587,1.074397e+06,2.613668e+05,9,2022,03
2,Alandroal,1977,2121,1.258384e+06,2.074845e+05,23,2022,03
3,Albergaria-a-Velha,5596,8525,4.441889e+06,1.406621e+07,52,2022,03
4,Albufeira,28711,24554,1.840090e+07,7.379642e+06,385,2022,03
...,...,...,...,...,...,...,...,...
95,Fundão,17512,6518,5.787765e+06,2.465204e+06,130,2022,03
96,Gavião,1863,1664,7.258333e+05,1.600605e+05,25,2022,03
97,Golegã,2619,1518,1.186724e+06,6.717029e+05,26,2022,03
98,Gondomar,37393,45385,2.583512e+07,7.833737e+06,405,2022,03


In [18]:
# aplicar one hot enconding no mes
# nao o fazemos no concelho pq isso aumentaria radicalmente a dimensao

ohenc = pd.get_dummies(merged_df, columns = ['Mes'], dtype=int)
ohenc.head()

,Concelho,Num_CPEs_Inteligentes,Num_CPEs_Nao_Inteligentes,Energia_Consumida_Baixa_T,Energia_Consumida_Superior_Baixa_T,Numero_total_ordens_servico,Ano,Mes_01,Mes_02,Mes_03,Mes_04,Mes_05,Mes_06,Mes_07,Mes_08,Mes_09,Mes_10,Mes_11,Mes_12
0,Abrantes,13331,11795,6.376950e+06,7.157803e+06,150,2022,0,0,1,0,0,0,0,0,0,0,0,0
1,Aguiar da Beira,2331,2587,1.074397e+06,2.613668e+05,9,2022,0,0,1,0,0,0,0,0,0,0,0,0
2,Alandroal,1977,2121,1.258384e+06,2.074845e+05,23,2022,0,0,1,0,0,0,0,0,0,0,0,0
3,Albergaria-a-Velha,5596,8525,4.441889e+06,1.406621e+07,52,2022,0,0,1,0,0,0,0,0,0,0,0,0
4,Albufeira,28711,24554,1.840090e+07,7.379642e+06,385,2022,0,0,1,0,0,0,0,0,0,0,0,0


In [19]:
train = ohenc[ohenc.Ano != '2024'].drop('Ano', axis = 1)
test = ohenc[ohenc.Ano == '2024'].drop('Ano', axis = 1)

In [20]:
X_train = train.drop(['Numero_total_ordens_servico', 'Concelho'], axis=1) #drop concelho pra poder usar KNN
X_test = test.drop(['Numero_total_ordens_servico', 'Concelho'], axis=1)

y_train = train.Numero_total_ordens_servico
y_test = test.Numero_total_ordens_servico

In [21]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6006 entries, 0 to 6005
Data columns (total 16 columns):
 #   Column                              Non-Null Count  Dtype  
---  ------                              --------------  -----  
 0   Num_CPEs_Inteligentes               6006 non-null   int64  
 1   Num_CPEs_Nao_Inteligentes           6006 non-null   int64  
 2   Energia_Consumida_Baixa_T           6006 non-null   float64
 3   Energia_Consumida_Superior_Baixa_T  6006 non-null   float64
 4   Mes_01                              6006 non-null   int64  
 5   Mes_02                              6006 non-null   int64  
 6   Mes_03                              6006 non-null   int64  
 7   Mes_04                              6006 non-null   int64  
 8   Mes_05                              6006 non-null   int64  
 9   Mes_06                              6006 non-null   int64  
 10  Mes_07                              6006 non-null   int64  
 11  Mes_08                              6006 non-nul

Normalização de Dados - Dados de Treino

In [22]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# num_cols = ['Num_CPEs_Inteligentes', 'Num_CPEs_Nao_Inteligentes', 'Energia_Consumida_Baixa_T', 'Energia_Consumida_Superior_Baixa_T']

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [23]:
from sklearn.neighbors import KNeighborsRegressor

knn3 = KNeighborsRegressor(3)
knn3.fit(X_train, y_train)

KNeighborsRegressor(n_neighbors=3)